# APRIORI ALGORITHM

In [94]:
#PD10_Kushagra Suryawanshi
from numpy import *

In [95]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

createC1: Creates C1

C1 is a candidate itemset of size one.

L1 meets minimum requirements. L1 then gets combined to become C2 and filtered to give L2. Frozensets afre immutable. 

In [96]:
#create empty list C1; create a list of single-items; sort the list; map list items to frozenlist and return it
def createC1(dataSet):
    C1=[]
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

In [97]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [98]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

scanD: takes 3 args: dataset, Ck: a list of candidate sets D, and minSupport

used to generate L1 from C1

it returns a dictionary with support values

ssCnt is a dictionary with support value for every item

In [99]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData
            
            

In [100]:
D = list(map(set, dataSet))

In [101]:
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [102]:
L1, suppDat0 = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [103]:
suppDat0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [104]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [105]:
C2 = aprioriGen(L1, 2)
C2

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [106]:
L2, supK = scanD(D, C2, 0.5)
supK.update(supK)
L2.append(L1)
L2

[frozenset({2, 3}),
 frozenset({3, 5}),
 frozenset({2, 5}),
 frozenset({1, 3}),
 [frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]]

In [107]:
C3 = aprioriGen(L2, 3)
C3

[frozenset({2, 3, 5})]

In [108]:
L3, supK = scanD(D, C3, 0.5)
supK.update(supK)
L3.append(L2)
L3

[frozenset({2, 3, 5}),
 [frozenset({2, 3}),
  frozenset({3, 5}),
  frozenset({2, 5}),
  frozenset({1, 3}),
  [frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]]]

In [109]:
C4 = aprioriGen(L3, 3)
C4

[]

In [110]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set,dataSet))
    L1,supportData = scanD(D,C1,0.5)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2],k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L,supportData

In [111]:
L,supportData = apriori(dataSet,0.5)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [112]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [113]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [114]:
L[2]

[frozenset({2, 3, 5})]

In [115]:
L[3]

[]

In [116]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [117]:
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [118]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [119]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf = 0.7):
    m = len(H[0])
    if(len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if(len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [120]:
L, supportData = apriori(dataSet, minSupport = 0.5)

In [121]:
rules = generateRules(L, supportData, minConf = 0.7)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0
